# L2 FWI Example
We will solve this problem using the following steps:
1. Prepare models
    * Read the true model from binary file
    * Generate our subsampled true and starting models 
2. Build up a small local compute cluster and load required packages 
3. Build F, the nonlinear distributed block operator for seismic modeling
4. Use F and the true model to create a synthetic data-set
5. Build the gradient and cost functions
6. Perform the FWI using optim.jl

## Add required packages
Uncomment the line below if you need to add these packages to your environment

In [ ]:
# ]add Optim LineSearches

In [ ]:
using PyPlot, Distributed, Jets, JetPack, JetPackDSP, Printf

## 1. Prepare models

#### Read true model from binary file

In [ ]:
v = read!("../20_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));

#### Subsample models
We will resample from the 20m grid to a 40m grid

In [ ]:
v = v[1:2:end,1:2:end]

dz,dx = 40.0,40.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

#### Smooth the true model to generate true and starting models
Note that we will perform this example at low frequency, and so we smooth both the true and the starting models before we begin. 

Please refer to the smoothing notebook in the `20_marmousi_model_setup` example directory for more details on our smoothing approach. Note that we apply the smoothing in slowness to preserve traveltime.

In [ ]:
# smoothing for the true model
ns1 = 1
P1 = JopPad(JetSpace(Float32,nz,nx), -ns1:nz+ns1, -ns1:nx+ns1, extend=true)
S1 = JopConvolve(range(P1), smoother=:rect, n=(ns1,ns1))
R1 = JopPad(JetSpace(Float32,nz,nx), -ns1:nz+ns1, -ns1:nx+ns1, extend=false)
s1 = R1' ∘ S1 ∘ P1 * (1 ./ v)
v1 = 1 ./(s1);

# smoothing for the starting model
ns2 = 5
P2 = JopPad(JetSpace(Float32,nz,nx), -ns2:nz+ns2, -ns2:nx+ns2, extend=true)
S2 = JopConvolve(range(P2), smoother=:rect, n=(ns2,ns2))
R2 = JopPad(JetSpace(Float32,nz,nx), -ns2:nz+ns2, -ns2:nx+ns2, extend=false)
s2 = R2' ∘ S2 ∘ P2 * (1 ./ v)
v2 = 1 ./(s2);

#### Replace the water layer in both models

In [ ]:
v1[v.==1500.0] .= 1500.0;
v2[v.==1500.0] .= 1500.0;

#### Visualize

In [ ]:
figure(figsize=(8,12))
vmin,vmax = extrema(v1)
dmin,dmax = -1000,1000
subplot(3,1,1); imshow(v1,aspect="auto",cmap="jet"); 
colorbar(); clim(vmin,vmax); title("True velocity (v1)")
subplot(3,1,2); imshow(v2,aspect="auto",cmap="jet");
colorbar(); clim(vmin,vmax); title("Starting velocity (v2)");
subplot(3,1,3); imshow(v1.-v2,aspect="auto",cmap="seismic");
colorbar(); clim(dmin,dmax); title("Difference (v1-v2)");

## 2. Build up a small local compute cluster and load required packages 

#### Setup OMP environment variables for the cluster
Note we need to do this because we are using multiple workers on the same physical node, and without setting up thread affinity the modeling will be *incredibly* slow.

In [ ]:
nthread = Sys.CPU_THREADS
ENV["OMP_DISPLAY_ENV"] = "true"
ENV["OMP_PROC_BIND"] = "close"
ENV["OMP_NUM_THREADS"] = "$(div(nthread,2))" 
addprocs(2)
@show workers()
for k in 1:nworkers()
    place1 = (k - 1) * div(nthread,nworkers())
    place2 = (k + 0) * div(nthread,nworkers()) - 1
    @show place1, place2, nthread
    @spawnat workers()[k] ENV["GOMP_CPU_AFFINITY"] = "$(place1)-$(place2)";
end

# ENV["OMP_DYNAMIC"] = "true"
# ENV["OMP_STACKSIZE"] = "64M"
# ENV["OMP_WAIT_POLICY"] = "active"
# ENV["OMP_DISPLAY_AFFINITY"] = "true"
# ENV["GOMP_CPU_AFFINITY"] = "0-$(nthread)"

In [ ]:
@everywhere using Distributed, DistributedArrays, DistributedJets, Jets, JetPack, WaveFD, JetPackWaveFD, LinearAlgebra, LineSearches, Optim, Random

## 3. Build F, the forward modeling operator

16 shot locations

In [ ]:
nshots = 12
sx = round.(Int,collect(range(0,stop=(nx-1)*dx,length=nshots)))
@show nshots;
@show sx;

#### Note on scratch space for temporary files
When dealing with serialized nonlinear wavefields as in this example, we need to specify the location where scratch files will be written.

You may need to change this to point to a temporary directory available on your system.

In [ ]:
@everywhere scratch = "/mnt/scratch"
@assert isdir(scratch)

In [ ]:
@everywhere begin
    ntrec = 2501
    dtrec = 0.004
    dtmod = 0.002
end

@everywhere function makeF(i,nz,nx,dz,dx,sx)
    JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = ntrec,
        dtrec = dtrec,
        dtmod = dtmod,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=4.0),
        sx = sx[i],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        nbz_cache = nz,
        nbx_cache = nx,
        comptype = UInt32,
        srcfieldfile = joinpath(scratch, "field-$i-$(randstring()).bin"),
        reportinterval=0)
end

In [ ]:
F = @blockop DArray(I->[makeF(i,nz,nx,dz,dx,sx) for i in I[1], j in I[2]], (nshots,1))

## 4. Use F to create a data for both models

In [ ]:
# This may take a while if running on a single node
t1 = @elapsed begin
    d1 = F*v1;
end
@show t1;
t2 = @elapsed begin
    d2 = F*v2;
end
@show t2;

#### Compute the residual data

In [ ]:
r = d1 .- d2;

#### Collect the data locally before plotting

In [ ]:
shots = [1:2:div(nshots,2);]
@show shots
d1local = [getblock(d1,i) for i in shots]
d2local = [getblock(d2,i) for i in shots]
rlocal = [getblock(r,i) for i in shots]
sxlocal = [sx[i] for i in shots]
@show sxlocal;

#### Plot shot gathers for true model, starting model, and residual
We flip the direction of the modeled data in order to help display the match at far offsets.

In [ ]:
scale = 10.0 / sqrt(norm(d1)^2 / length(d1))
shots = [1:2:div(nshots,2);]
@show scale
@show shots

figure(figsize=(16,24)); clf()
for (iplot,ishot) in enumerate(shots)
    subplot(3,3,(iplot-1)*3+1);
    imshow(d1local[iplot],cmap="gray",aspect="auto",clim=[-1,+1]);
    title("True Model $(ishot) at x=$(sx[ishot])");
    subplot(3,3,(iplot-1)*3+2);
    imshow(reverse(d2local[iplot],dims=2),cmap="gray",aspect="auto",clim=[-1,+1]);
    title("Starting Model $(ishot) at x=$(sx[ishot])");
    subplot(3,3,(iplot-1)*3+3);
    imshow(10 .* rlocal[iplot],cmap="gray",aspect="auto",clim=[-1,+1]);
    title("Residual $(ishot) at x=$(sx[ishot]) x10");
end
tight_layout()

#### Asses if data is cycle skipped at the farthest offsets
Next we plot the far offset traces for these shots to determine if the data is cycle skipped. 

You can see in the plots below that the refraction waveforms (first arrivals) in the starting model are not cycle skipped, so we proceed. 

Note that a very significant part of the residual wavefield is actually reflections in this example.

In [ ]:
scale = 10.0 / sqrt(norm(d1)^2 / length(d1))
t = [0.0:dtrec:dtrec*(ntrec - 1);]

figure(figsize=(12,8)); clf()
for (iplot,ishot) in enumerate(shots)
    subplot(3,1,iplot);
    plot(t,d1local[iplot][:,end],label="True Model $(ishot) at x=$(sx[ishot])");
    plot(t,d2local[iplot][:,end],label="Starting Model $(ishot) at x=$(sx[ishot])");
    xlim([4.5,t[end]])
    legend()
end
tight_layout()

## 5. Build the gradient and cost functions

#### Build water bottom mask for gradient
We use this mask to remove imprint in gradient very close to the source locations. The mask is set to 0 wherever the true velocity model has value 1500 m/s, and is set to 1 otherwise. 

In [ ]:
wb_mask = ones(Float32,size(v))
wb_mask[v.==1500.0] .= 0;
figure(figsize=(6,3))
imshow(wb_mask,aspect="auto",cmap="gray_r",clim=[0,+2]);
colorbar();
title("Water Bottom Mask");

#### Gradient function
This method is called by the solver whenever the gradient is required. Steps in computing the gradient are as follows:
1. Compute the source side illumination
1. Establish the illumination compensation preconditioner as `1/illum^2`
1. Apply the adjoint of the Jacobian to the current residual `d - F*v` 
1. Apply very simple scaling based on the size of the first gradient and save to apply to future gradients

In [ ]:
function gradient!(G,F,v,dobs,p,wb_mask)
    J = jacobian(F,v)
    s = srcillum(J)
    s .= ((s ./ maximum(s)).^2) .+ 1e-8
    R = JopDiagonal((1 ./ s).^2)
    t = @elapsed begin
        dv = F*v
        G .= R' ∘ J' * (dv .- dobs)
    end
    phi = norm(dvϕdobs)
    p.count += 1
    @info @sprintf("%3d g time,g rms,cost; %10.4e %10.4e %10.4e\n", 
        p.count, t, sqrt(norm(G)^2 / length(G)), phi)
    G .*= wb_mask #mute water column
    if p.gscale == 0.0
        # compute scalar from first gradient, apply to future gradients
        p.gscale = 10 ./ maximum(G) 
    end
    G .*= p.gscale
end
# struct to save the first gradient scalar
mutable struct FwiPar
   count::Int64
   gscale::Float64
end
p = FwiPar(0,0.0)
g!(G,x) = gradient!(G,F,x,d1,p,wb_mask)

#### Compute the initial gradient and display

In [ ]:
grad = zeros(Float32,size(v1))
tgrad = @elapsed begin
    g!(grad,v2)
end
@show tgrad;

In [ ]:
figure(figsize=(12,6))
dv = v1 .- v2
subplot(1,2,1)
imshow(-1 .* grad ./ maximum(abs,grad),aspect="auto",cmap="seismic");
colorbar(orientation="horizontal");clim(-1,1);title("-1 * Gradient");
subplot(1,2,2)
imshow(dv ./ maximum(abs,dv),aspect="auto",cmap="seismic");
colorbar(orientation="horizontal");clim(-1,1);title("Velocity Difference: (v - vₒ)");

In [ ]:
function cost(x,F,d1)
    dx = F*x
    phi = norm(d1 .- dx)
    return phi
end
f(x) = cost(x,F,d1)

In [ ]:
@info "initial cost; $(f(v2))";

## 6. Do the FWI using optim.jl
We do 10 iterations of limited memory BFGS

In [ ]:
niter = 25
solver = LBFGS(m = niter, 
    alphaguess = LineSearches.InitialQuadratic(), 
    linesearch = LineSearches.MoreThuente());

In [ ]:
function mycallback(state::OptimizationState)
    @info "iter=$(state.iteration), cost=$(state.value), |grad|=$(state.g_norm)"
    false
end
mycallback(trace::OptimizationTrace) = mycallback(trace[end]);

In [ ]:
p = FwiPar(0,0.0)
redirect_stdout(open("/dev/null","w")) # otherwise the models are printed to stdout at every iteration

topt = @elapsed begin
    result = optimize(f, g!, v2, solver,
        Optim.Options(
            iterations = niter,
            show_trace = true,
            store_trace = true,
            show_every = 1,
            extended_trace = true,
            allow_f_increases = false,
            callback = mycallback))
end

In [ ]:
@sprintf("Time for optimization; %.2f minutes\n", topt / 60)

In [ ]:
vfwi = Optim.minimizer(result) # optimal solution
ϕ = Optim.f_trace(result)      # cost vs iteration
m = Optim.x_trace(result);     # model vs iteration

## Display the velocity models 

In [ ]:
figure(figsize=(9,12)); clf()

subplot(3,1,1);imshow(v1,aspect="auto",cmap="jet");
colorbar(orientation="vertical");clim(vmin,vmax);title("True velocity")

subplot(3,1,2);imshow(v2,aspect="auto",cmap="jet");
colorbar(orientation="vertical");clim(vmin,vmax);title("Starting velocity");

subplot(3,1,3);imshow(vfwi,aspect="auto",cmap="jet");
colorbar(orientation="vertical");clim(vmin,vmax);title("FWI velocity");

## Display the velocity difference models 

In [ ]:
rms_v2   = @sprintf("%.1f m/s", sqrt(norm(v1 .- v2)^2 / length(v1)))
rms_vfwi = @sprintf("%.1f m/s", sqrt(norm(v1 .- vfwi)^2 / length(v1)))

figure(figsize=(9,8)); clf()

subplot(2,1,1);imshow(v1 .- v2,aspect="auto",cmap="seismic");
colorbar(orientation="vertical");clim(dmin,dmax);
title("Vtrue - Vstart difference, rms=$(rms_v2)");

subplot(2,1,2);imshow(v1 .- vfwi,aspect="auto",cmap="seismic");
colorbar(orientation="vertical");clim(dmin,dmax);
title("Vtrue - Vfwi difference, rms=$(rms_vfwi)");


## Display the cost function 

In [ ]:
plot(ϕ ./ ϕ[1])
xlabel("Nonlinear Iteration")
ylabel("Normalized cost")
title("FWI Objective Function ||f(v) - d||");

## Clean up the local disk and remove the workers

In [ ]:
close(F)
# rmprocs(workers());